In [2]:
!pip install sympy matplotlib numpy scipy IPython itikz 

In [3]:
%load_ext itikz
%matplotlib notebook

In [4]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
r, beta, alpha, omega1 = sp.symbols('r beta alpha omega1')
T1 = sp.Function('T1')
T2 = sp.Function('T2')
phi = sp.Function('phi')
varphi = sp.Function('varphi')

from IPython.display import display, Math,Latex
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sp.Number)})


<img src="imagenes/freno_esquema.png" width=50%>

In [22]:
b = 5 *1e-3
N = 3600 
omega = N*1/60*2*np.pi
mu = 0.15 
P = 1 *1e6
k = 75 
Rd = 4.0 *1e-2
Rp = 3.0 *1e-2
h = 20+1500 *(omega1/100)**1.25

alpha_i = 2*P*mu*omega/b/k
beta_i = (2*h/k/b)**.5
Tinf = 30 

Balance térmico en disco en contacto con pastilla de freno
$$\frac{d}{dr}(-k\frac{dT}{dr}2\pi r b)dr = 4\pi r^2p\mu\omega dr$$

In [23]:
equation2 = sp.Derivative((r*sp.Derivative(T2(r),r,evaluate=False)),r)+r**2*alpha 
display(Latex(r'Para $r>R_p \\ \displaystyle'+ sp.latex(equation2)+'=0$'))
display(Latex(r'$\displaystyle\alpha=2\frac{p\mu\omega}{k b}$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [24]:
T2f = sp.dsolve(equation2).args[1]
T2f = T2f.subs('C2','A1').subs('C1','A2')
#T2f = T2f.subs('C1','C2')
display(T2f)


#condicion de borde aislado
display(Latex(r'Condicion de borde aislado $\displaystyle\left.\frac{dT_2}{dr}\right|_{r=R_d}=0$'))
#C2i = sp.solve(T2f.diff('r').subs('r',Rd).subs('alpha',alpha_i))[0]
display(Latex(r'$\displaystyle'+sp.latex(T2f.diff(r))+'=0$'))
#T2f = T2f.subs('C2',C2i)
A1 = sp.solve(T2f.diff(r),'A1')

A1*log(r) + A2 - alpha*r**3/9

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


<img src="imagenes/freno_calor.png" width=20%>


Para la región $r<R_p$:

\begin{eqnarray}
\dot q|_{r+dr}d\theta(r+dr) - \dot q|_r d\theta rb& =& 2h dr (rd\theta) \Delta T\\
\end{eqnarray}
$\dot q|_{r+dr}b =  \dot q|_r + \partial \dot q/\partial r|_r dr $

\begin{eqnarray}
(\dot q|_r dr+ \partial \dot q/\partial r|_r r )   b&=& 2h dr \Delta T \\
\partial/\partial r(r   \dot q_r)  dr b&=&  2h r dr \Delta T
\end{eqnarray}
Sea $\varphi = T-T_\infty$ y $\beta^2 = 2h/kb$
\begin{equation}
\frac{\partial}{\partial r}\left(-r \frac{\partial \varphi}{\partial r}\right) = \beta^2 r \varphi
\end{equation}

Para $\varphi = T - T_\infty$.

In [25]:
equation1 = sp.Derivative((r*sp.Derivative(varphi(r),r,evaluate=False)),r)-beta**2*r*varphi(r) 
display(Latex(r'La ecuación $\displaystyle'+ sp.latex(equation1)+'=0$'))
phi1 = sp.dsolve(equation1).args[1]
phi1 = phi1.subs('C1','B1').subs('C2','B2')
display(Latex(r'Tiene solución general, $\varphi (r) = '+sp.latex(phi1)+'$'))
display(Latex(r'Solución finita en $r=0, \Longrightarrow \varphi(r=0)='+sp.latex(phi1.subs('r',0))+'$'))
display(Latex(r'$\Longrightarrow \varphi(r)='+sp.latex(phi1.subs('B2',0))+'$'))
phi1 = phi1.subs('B2',0)
T1 = phi1+Tinf
display(Latex(r'Calor en $\displaystyle r=R_p,\\ \displaystyle-k\frac{d\varphi}{dr}=-k\frac{dT_2}{dr}$'))
B1 = sp.solve(-T2f.diff(r)+phi1.diff(r),'B1')
display(Latex('$B_1= '+sp.latex(B1i[0])+'$'))


display(Latex(r'Temperaturas en $\displaystyle r=R_p,\\T_1=T_2$'))

A2 = sp.solve(T2f-T1,'A2')
display(Latex(r'$A_2='+sp.latex(A2i)+'$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

NameError: name 'B1i' is not defined

In [26]:
valor_A1 = A1[0].subs('alpha',alpha_i).subs('r',Rd)
valor_B1 = sp.N(B1[0].subs('r',Rp).subs('A1',valor_A1).subs('beta',beta_i).subs('alpha',alpha_i))
valor_A2 = A2[0].subs('r',Rp).subs('A1',valor_A1).subs('beta',beta_i).subs('alpha',alpha_i).subs('B1',valor_B1)

Temp = sp.Piecewise( (T1, r<Rp), (T2f, r>=Rp) )

NameError: name 'A2' is not defined

In [9]:
T1_fun = sp.lambdify([r],T1.subs('beta',beta_i).subs('alpha',alpha_i).subs('B1',valor_B1).subs('omega1',omega))

dT1dr_fun = sp.lambdify([r],T1.diff('r').subs('beta',beta_i).subs('alpha',alpha_i).subs('B1',valor_B1).subs('omega1',omega))
T2_fun = sp.lambdify([r],T2f.subs('A1',valor_A1).subs('alpha',alpha_i).subs('A2',valor_A2).subs('omega1',omega))

Temp_fun = sp.lambdify([r],Temp.subs('A1',valor_A1).subs('alpha',alpha_i).subs('A2',valor_A2).subs('beta',beta_i).subs('B1',valor_B1).subs('omega1',omega))


NameError: name 'valor_B1' is not defined

In [10]:
fig0,ax0 = plt.subplots()
r1 = np.linspace(0,Rp,100)
r2 = np.linspace(Rp,Rd,100)
r_i = np.linspace(1e-6,Rd,30)
ax0.plot(r1,T1_fun(r1))
ax0.plot(r2,T2_fun(r2))
ax0.plot(r_i,Temp_fun(r_i),'o',fillstyle='none')
ax0.set_xlabel('$r[m]$')
ax0.grid()
ax0.set_ylabel('$T[^\circ$C]');

<>:10: SyntaxWarning: invalid escape sequence '\c'
<>:10: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_1419937/1159545971.py:10: SyntaxWarning: invalid escape sequence '\c'
  ax0.set_ylabel('$T[^\circ$C]');


<IPython.core.display.Javascript object>

/tmp/ipykernel_1419937/1159545971.py:10: SyntaxWarning: invalid escape sequence '\c'
  ax0.set_ylabel('$T[^\circ$C]');


NameError: name 'T1_fun' is not defined

In [11]:
def h_conv(omega):
    return 20+1500 *(omega/100)**1.25
def alpha_fun(omega):
    return 2*P*mu*omega/b/k
def beta_fun(h):
    return (2*h/k/b)**.5


In [12]:
N = 3600
Ns = np.linspace(3600,3800,2)
omegas = Ns/60*2*np.pi
Tmax_i = np.zeros_like(omegas)
for i, omegai in enumerate(omegas):    
    h_convi = h_conv(omegai)
    valor_A1 = A1[0].subs('r',Rd).subs('alpha',alpha_fun(omegai))
    valor_B1 = B1[0].subs('A1',valor_A1).subs('alpha',alpha_fun(omegai)).subs('beta',beta_fun(h_convi)).subs('r',Rp).evalf()
    valor_A2 = A2[0].subs('r',Rp).subs('A1',valor_A1).subs('beta',beta_fun(h_convi)).subs('alpha',alpha_fun(omegai)).subs('B1',valor_B1).subs('omega1',omegai)
    Temp_fun2 = sp.lambdify([r],Temp.subs('A1',valor_A1).subs('alpha',alpha_fun(omegai)).subs('A2',valor_A2).subs('beta',beta_fun(h_convi)).subs('B1',valor_B1).subs('omega1',omegai))

    Tmax_i[i] = Temp_fun2(r_i).max()

NameError: name 'A1' is not defined

In [13]:
valor_A2.evalf()
valor_B1.evalf()

NameError: name 'valor_A2' is not defined

In [14]:
B1

NameError: name 'B1' is not defined

In [15]:
fig1,ax1 = plt.subplots()
ax1.plot(Ns,Tmax_i,'o')


<IPython.core.display.Javascript object>

In [16]:
alpha_fun(omega)

301592894.7446201

In [17]:
alpha_i

301592894.7446201

In [18]:
fig0,ax0 = plt.subplots()
ax0.plot(r_i,Temp_fun2(r_i))

<IPython.core.display.Javascript object>

NameError: name 'Temp_fun2' is not defined

In [129]:
Temp

Piecewise((B1*besselj(0, r*sqrt(-beta**2)) + 30, r < 0.03), (A1*log(r) + A2 - alpha*r**3/9, True))

alpha/3

In [285]:
beta_i

10.3279555898864*(0.237170824512628*omega1**1.25 + 1)**0.5

alpha/3

In [334]:
phi_1, phi_2, b_1, b_2, phi_f, b = sp.symbols('phi_1 phi_2 b_1 b_2 phi_f b ')

\begin{eqnarray}
\frac{\phi_1}{\phi_2}& = & \frac{b_1}{b_2}\\
\frac{\phi_1}{\phi_2}+1& = & \frac{b_1}{b_2}+1\\
\frac{\phi_1}{\phi_2}+\frac{\phi_2}{\phi_2}& = & \frac{b_1}{b_2}+\frac{b_2}{b_2}\\
\frac{\phi_1+\phi_2}{\phi_2}& = & \frac{b_1+b_2}{b_2}\\
\frac{\phi_1+\phi_2}{b_1+b_2}& = & \frac{\phi_2}{b_2}\\
\end{eqnarray}